In [ ]:
import pandas     as pd
import matplotlib as plt
import pmdarima   as pm
import statsmodels.api   as sm
import matplotlib.pyplot as plt
import warnings

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA

from   datetime                    import datetime
from   dateutil                    import relativedelta
from   pmdarima.arima              import auto_arima
from   pmdarima.arima              import ndiffs
from   sklearn.model_selection     import train_test_split
from   sklearn.metrics             import mean_squared_error as mse
from   arch import arch_model
from   tqdm import tqdm

# Data

In [ ]:
pred_year  = 35
pred_month = pred_year * 12
data_start = "1997-01"
data_end   = "2023-05"
test_start = '2009-01'
start_pt   = pd.date_range(start=test_start, end=data_end, freq="MS").strftime("%Y-%m").tolist()

df         = pd.read_csv("/Users/wangshihhung/Desktop/ARMA/house_cb.csv")
df["date"] = pd.to_datetime(df["date"], format="%YM%m").dt.strftime('%Y-%m')
df.index   = df["date"]

# Stationarity Test (KPSS)

In [ ]:
Ut = df['rate'][df["date"] < test_start]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
plot_acf (Ut, ax=axes[0])
plot_pacf(Ut, ax=axes[1])
plt.show()

From ACF and PACF plot, we cannot observe any obvious cutoffs for lags.

The Kwiatkowski–Phillips–Schmidt–Shin (KPSS) test is commonly used to test for stationarity in time series data. Unlike the ADF test, KPSS directly tests for stationarity, assuming the series is stationary under the null hypothesis.

H₀ (null): The series is stationary (no unit root)

H₁ (alt): The series is non-stationary (has a unit root)

Here, we choose 𝛼=0.05.

If the p-value of the KPSS test is smaller than  𝛼
 , we reject the null hypothesis, suggesting the time series data is non-stationary.

We need to further difference the series and run the test again.

In [ ]:
statistic, p_value, _, _ = kpss(Ut, regression='c', nlags='auto')
print("p-valus from KPSS test : {:.4f}".format(p_value))

if p_value > 0.05:
    Vt = Ut
    d = 0
else:
    Vt = np.diff(Ut)
    d = 1
    while True:
        statistic, p_value, _, _ = kpss(Vt, regression='c', nlags='auto')
        if p_value > 0.05 or len(Vt) <= 10:
            break
        Vt = np.diff(Vt)
        d += 1

In [ ]:
print("The order of difference is {:d}".format(d))

The test statistic is outside of the range of p-values available in the look-up table. The actual p-value is greater than the p-value returned.

What does this mean?

The test statistic from the KPSS test is too small, falling outside the range of the lookup table built into statsmodels.

Since the p-value is obtained by referencing this table, and your statistic is smaller than the minimum available, it means the true p-value is actually larger than the one returned (for example, larger than 0.1).

# ARMA model

Due to computational and time constraints, we set max_p and max_q in advance.

Also, we use AIC for model evaluation. Smaller AIC indicates better model performance.

In [ ]:
max_p = 10
max_q = 10

In [ ]:
warnings.filterwarnings("ignore")

AIC = np.inf
ARMA_order = None
ARMA_model = None

for p in tqdm(range(max_p+1)):
    for q in range(max_q+1):
        try:
            model = ARIMA(Vt, order=(p, 0, q)).fit(method='statespace')
            if model.aic < AIC:
                AIC = model.aic
                ARMA_order = (p, q)
                ARMA_model = model
        except:
            continue


warnings.filterwarnings("default")

In [ ]:
print("Through AIC model selection, the optimal AR and MA orders are {:d} and {:d}".format(ARMA_order[0], ARMA_order[1]))

In [ ]:
Et = ARMA_model.resid

The Ljung-Box test is commonly used to assess whether the residuals of a time series model are independently distributed. It specifically tests for the presence of autocorrelation at multiple lags.

**H₀ (null): The residuals are independently distributed (no autocorrelation, i.e., white noise).**

**H₁ (alt): The residuals are not independently distributed (presence of autocorrelation).**



1. If the p-value of the Ljung-Box test is smaller than $\alpha$, we reject the null hypothesis, suggesting that the residuals exhibit autocorrelation.

2. If the p-value is larger than alpha, we fail to reject the null hypothesis, suggesting that the residuals are consistent with white noise.

Here, we choose $$\alpha = 0.05$$

In [ ]:
lb_test = acorr_ljungbox(Et, lags=range(1, 16), return_df=True)

print(lb_test)

From Ljung-Box test, we may conclude that, $E_{t}$ is white noise process.

# ARCH effect

Engle’s test: 
Check whether the residuals from your ARMA model have time-varying volatility (exhibits ARCH effects).

**H₀ (null): No ARCH effect — the residual variance is constant over time**

**H₁ (alt): ARCH effect is present — variance depends on past squared residuals**
    
    
Here, we choose $$\alpha = 0.05$$.

**If the p-value of the Engle’s test is larger than alpha, we cannot reject null hypothesis, suggesting there is No ARCH effect.**

**Otherwise, we need to fit GARCH model.**

In [ ]:
def GARCH_fit(Et, max_p=5, max_q=5):
    AIC = np.inf
    GARCH_model = None
    GARCH_order = None

    max_p, max_q = 5, 5

    for p_val in range(1, max_p + 1):
        for q_val in range(1, max_q + 1):
            print("loop...")
            try:
                model = arch_model(Et, vol='GARCH', p=p_val, q=q_val, mean = 'Zero', rescale=True)
                fit   = model.fit(disp='off')

                if fit.aic < AIC:
                    AIC = fit.aic
                    GARCH_model = fit
                    GARCH_order = (p_val, q_val)                
            except:
                continue

    if GARCH_model is None:
        print("Hi!!!!")
        model = arch_model(Et, vol='GARCH', p=1, q=1, mean = 'Zero', rescale=True)
        GARCH_model = model.fit(disp='off')
        GARCH_order = (1, 1)
        
    #sigma_t = GARCH_model.conditional_volatility**2
    
    return GARCH_model, GARCH_order #, sigma_t

In [ ]:
from statsmodels.stats.diagnostic import het_arch

stat, pval, _, _ = het_arch(Et, nlags=10)

if pval < 0.05:
    GARCH_model, GARCH_order = GARCH_fit(Et, max_p=5, max_q=5)
else:
    GARCH_model, GARCH_order = None, None
    sigma_t_hat = np.std(Et) * np.ones(len(Et))

In [ ]:
print(pval)

\newpage

# ARIMAX model

In [ ]:
AIC          = np.inf
ar, diff, ma = 0, 0, 0
AIC_val      = {}

model_para = {}
model_rate = pd.DataFrame({'date':pd.date_range(start = data_start, end = "2058-05", freq="MS")})
model_rate.index = model_rate["date"]

In [ ]:
start_pt = ["2009-01", "2010-01", "2011-01", "2012-01", "2013-01", "2014-01", "2015-01", "2016-01", 
            "2017-01", "2018-01", "2019-01", "2020-01", "2021-01", "2022-01", "2023-01"]
for month in tqdm( start_pt ):
    temp = df[ df["date"] < month ]
    
    for d in [0,1,2]:
        for p in range(11):
            for q in range(11):
                model3   = sm.tsa.arima.ARIMA(endog=temp['rate'], exog=temp[['rediscount']],order=[p,d,q], trend='n', freq='MS')
                results3 = model3.fit()

                if AIC > results3.aic:
                    AIC = results3.aic
                    ar, diff, ma = p, d, q
                    print("Better!")

    #AIC_val[(p, d, q)] = results3.aic
    
    test_end = str( pd.to_datetime(month) + pd.DateOffset(months=pred_month - 1) )
    #delta     = relativedelta.relativedelta(pd.to_datetime(test_end), pd.to_datetime(data_end))
    #exog     = [ df.at[month, "rediscount"] ] * ( delta.years*12 + delta.months )
    ex     = [ df.at[month, "rediscount"] ] * pred_month
    
    model    = sm.tsa.arima.ARIMA(endog=temp['rate'],exog=temp[['rediscount']], order=[ar, diff, ma], trend='n', freq='MS')
    results  = model.fit()
    
    model_val  = results.predict (start=data_start, end=test_end, exog=ex)
    model_rate = model_rate.merge(model_val, left_index=True, right_index=True, how="left" )
    model_para[f"{month}"] = (ar, diff, ma)
    

In [ ]:
model   = sm.tsa.ARIMA(endog=df['rate'],exog=df[['rediscount']],order=[ar, diff, ma],trend='n')
results = model.fit()

In [ ]:
ex = []
for i in range(pred_month):
    ex.append(1.875)
results.predict(start="2023-06-01", end="2058-05-01", exog=ex).to_csv("/Users/wangshihhung/Desktop/ARMA/ARIMAX.csv", index=True)